In [12]:
import tensorflow as tf
print(len(tf.config.list_physical_devices('GPU')))

0


In [9]:
import os
import librosa
import numpy as np
import openl3
from sklearn.metrics import silhouette_score
from tqdm import tqdm

In [8]:
paths = {
    "drones": "../data/drones",
    "not_drones": "../data/not_drones"
}

files, labels = [], []
for label, path in paths.items():
    for root, _, fs in os.walk(path):
        for f in fs:
            if f.lower().endswith((".wav", ".mp3")):
                files.append(os.path.join(root, f))
                labels.append(1 if label == "drones" else 0)

print(f"{len(files)} файлов")


70626 файлов


In [10]:
embeddings = []

for f in tqdm(files, desc="формируем эмб", unit="файл"):
    audio, sr = librosa.load(f, sr=None, mono=True)
    emb, _ = openl3.get_audio_embedding(audio, sr, embedding_size=512, content_type="env")
    embeddings.append(np.mean(emb, axis=0))

X = np.array(embeddings)
y = np.array(labels)

score = silhouette_score(X, y)
print(f"\nSilhouette score (OpenL3): {score:.3f}")

формируем эмб:   0%|          | 0/70626 [00:00<?, ?файл/s]

2/2 [==============================] - 3s 866ms/step


формируем эмб:   0%|          | 1/70626 [00:13<256:04:38, 13.05s/файл]

2/2 [==============================] - 3s 847ms/step


формируем эмб:   0%|          | 2/70626 [00:16<144:31:53,  7.37s/файл]

...

2/2 [==============================] - 3s 876ms/step


формируем эмб:   0%|          | 6/70626 [00:29<75:58:15,  3.87s/файл]

1/2 [==============>...............] - ETA: 2s

формируем эмб:   0%|          | 6/70626 [00:33<108:17:34,  5.52s/файл]


KeyboardInterrupt: 

In [ ]:
np.savez("../embeddings/openl3_embeddings.npz", X=X, y=y)

with open("../RESULTS.md", "a") as f:
    f.write(f"| OpenL3 | {score:.3f} |\n")